# Proyecto Final — Notebook Integrado

Este cuaderno combina **Parte 1 (Preprocesamiento)** y **Parte 2 (Modelado y Evaluación)** en un solo flujo reproducible.

- Fuente 1: `ProyectoFINAL_Pt1.ipynb`
- Fuente 2: `ProyectoFINAL_COMPLETO.ipynb`

> Sugerencia: ejecuta las celdas en orden; todas las salidas fueron limpiadas para evitar confusiones.

## 📣 Pronóstico de tasa de cancelación de clientes - Telecom

# 📣 Pronóstico de tasa de cancelación de clientes - Telecom

## 📌 Introducción
Al operador de telecomunicaciones Interconnect le gustaría poder pronosticar su tasa de cancelación de clientes. 
    
Si se descubre que un usuario o usuaria planea irse, se le ofrecerán códigos promocionales y opciones de planes especiales. 
 

## 🎯 Problema de negocio
La clientela puede elegir entre un pago mensual o firmar un contrato de 1 o 2 años. Puede utilizar varios métodos de pago y recibir una factura electrónica después de una transacción. Telecom necesita descubrir si un usuario o usuaria planea irse, para ofrecerle códigos promocionales y opciones de planes especiales. 


## 🔍 Objetivo del proyecto
✔ Preparar un *snapshot* al **01-feb-2020** y un dataset limpio para modelado donde la **clase positiva** sea **`stay = 1`** (cliente activo, `EndDate == "No"`).  


## Inicialización

In [ ]:
# Cargar todas las librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
import numpy as np

In [ ]:
df_contract = pd.read_csv("./datasets/contract.csv")
df_personal = pd.read_csv("./datasets/personal.csv")
df_internet = pd.read_csv("./datasets/internet.csv")
df_phone    = pd.read_csv("./datasets/phone.csv")

## 📌 1. Descripción y preprocesamiento de los datos 

## 🎯 1.1  Contract

In [ ]:
print('Dimensiones:', df_contract.shape)
print('\nTipos de datos:')
df_contract.info()


In [ ]:
df_contract.head(5)

### 🧹 1.1.1 Convertir fechas a datetime

In [ ]:
df_contract["BeginDate"] = pd.to_datetime(df_contract["BeginDate"])
df_contract["EndDate"]   = pd.to_datetime(df_contract["EndDate"], errors="coerce")  
df_contract["MonthlyCharges"] = pd.to_numeric(df_contract["MonthlyCharges"], errors="coerce")
df_contract["TotalCharges"]   = pd.to_numeric(df_contract["TotalCharges"],   errors="coerce")

### 🧹 1.1.2 Tenure al snapshot

In [ ]:
snapshot = pd.Timestamp("2020-02-01")
df_contract["tenure_months"] = (((snapshot - df_contract["BeginDate"]).dt.days)
                             .clip(lower=0) / 30.44).round().astype(int)

### 🛑 1.1.3 Etiquetas

In [ ]:
df_contract["churn"] = df_contract["EndDate"].notna().astype(int)   # 1 = se va
df_contract["stay"]  = df_contract["EndDate"].isna().astype(int)    # 1 = se queda (objetivo)

In [ ]:
df_contract.head(5)

## 🎯 1.2 Personal

In [ ]:
print('Dimensiones:', df_personal.shape)
print('\nTipos de datos:')
df_personal.info()

In [ ]:
df_personal.head(5)

## 🎯 1.3 Internet

In [ ]:
print('Dimensiones:', df_internet.shape)
print('\nTipos de datos:')
df_internet.info()


In [ ]:
df_internet.head(5)

## 🎯 1.4 Phone

In [ ]:
print('Dimensiones:', df_phone.shape)
print('\nTipos de datos:')
df_phone.info()


In [ ]:
df_phone.head(5)

## 🖇️ 1.5 Unión por costumerID

In [ ]:
df = (df_contract
      .merge(df_personal, on="customerID", how="left")
      .merge(df_internet, on="customerID", how="left")
      .merge(df_phone,    on="customerID", how="left"))

In [ ]:
df.head(10)

In [ ]:
df.info()

## 📍 1.6 Indicadores y ausencias de servicio

In [ ]:
df["HasInternet"] = df["InternetService"].notna().astype(int)
df["HasPhone"]    = df["MultipleLines"].notna().astype(int)

internet_cols = ["InternetService","OnlineSecurity","OnlineBackup",
                 "DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]
for col in internet_cols:
    df[col] = df[col].fillna("No")
df["MultipleLines"] = df["MultipleLines"].fillna("No")

## ✏️ 1.7 Normalización de categorías

In [ ]:
n_cols = ["PaperlessBilling","Partner","Dependents"] + internet_cols + ["MultipleLines"]
for col in n_cols:
    if col in df:
        df[col] = (df[col].astype(str).str.strip()
                 .replace({"No internet service":"No", "No phone service":"No"}))

In [ ]:
# Imputación mínima
na_mask = df["TotalCharges"].isna()
df.loc[na_mask, "TotalCharges"] = (df.loc[na_mask, "MonthlyCharges"] * df.loc[na_mask, "tenure_months"]).round(2)


In [ ]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
print("Nulos antes:", df["TotalCharges"].isna().sum())

na_mask = df["TotalCharges"].isna()
df.loc[na_mask, "TotalCharges"] = (df.loc[na_mask, "MonthlyCharges"] * df.loc[na_mask, "tenure_months"]).round(2)

print("Nulos después:", df["TotalCharges"].isna().sum())  # debería ser 0


In [ ]:
# Selección de columnas
df_keep = ["customerID","stay","churn","MonthlyCharges","TotalCharges","tenure_months",
        "Type","PaperlessBilling","PaymentMethod","gender","SeniorCitizen","Partner","Dependents",
        "HasInternet","HasPhone","InternetService","OnlineSecurity","OnlineBackup",
        "DeviceProtection","TechSupport","StreamingTV","StreamingMovies","MultipleLines"]


In [ ]:
df_model = df[df_keep].copy()

print("Shape final:", df_model.shape)
print("Proporción stay:", round(df_model["stay"].mean(), 4))
print("Proporción churn:", round(df_model["churn"].mean(), 4))
display(df_model.head())


In [ ]:
clean_csv = "data_clean.csv"
df_model.to_csv(clean_csv, index=False)
print(f"Archivo guardado: {clean_csv}")

## 📌 2. EDA

In [ ]:
(df_model.isna().sum()
 .sort_values(ascending=False)
 .to_frame("nulos"))

## 2.1 🔒 Stay Rate

In [ ]:
for col in ["Type","PaymentMethod","InternetService","OnlineSecurity","TechSupport","MultipleLines"]:
    print(f"\n>>> {col}")
    display((df_model.groupby(col)["stay"]
             .mean()
             .sort_values(ascending=False)
             .round(3)
             .to_frame("stay_rate")))

## 📊 2.2 Gráficos

### 2.2.1 Distribuciones

In [ ]:
for col in ["MonthlyCharges","TotalCharges","tenure_months"]:
    plt.figure()
    df_model[col].dropna().hist(bins=30)
    plt.title(f"Distribución de {col}")
    plt.xlabel(col); plt.ylabel("Frecuencia")
    plt.show()

#### Conclusiones

1) MonthlyCharges

La distribución muestra que, a medida que la factura mensual sube, es más común que la permanencia baje.

Sugerencia: vigilar aumentos de precio y ofrecer planes escalonados o beneficios a quienes pagan más.

2) TotalCharges

En la distribución se puede ver que los clientes con un alto acumulado tienden a seguir con la empresa, lo que prueba que cuanto más tiempo dentro, más se estabiliza la relación.

Sugerencia: cuidar la experiencia temprana porque, una vez superada la etapa inicial, la permanencia mejora sola.

3) tenure_months 

Se puede observar que los clientes que se quedan se agrupan en antigüedades medias y altas, mientras que quienes se van se concentran en los primeros meses.

Sugerencia: dar seguimiento en los primeros meses.


### 2.2.2 Tasa de clientes que se quedan

In [ ]:

def bar_rate(df, cat_col, target='stay'):
    rates = (df.groupby(cat_col)[target].mean().sort_values(ascending=False))
    plt.figure()
    rates.plot(kind="bar")
    plt.title(f"Tasa de permanencia por {cat_col}")
    plt.ylabel(f"{target}_rate"); plt.xlabel(cat_col)
    plt.ylim(0, 1); plt.xticks(rotation=45, ha="right")
    plt.tight_layout(); plt.show()

for col in ["Type","PaymentMethod","InternetService","OnlineSecurity","TechSupport","MultipleLines"]:
    bar_rate(df_model, col, target="stay")

#### Conclusiones

1) Type (tipo de contrato)

En el gráfico se muestra que el contrato mensual es el segmento de mayor riesgo; los contratos anuales y bienales retienen mucho mejor.

Sugerencia: ofrecer un incentivo para la retención de los clientes.

2) PaymentMethod

Se puede observar que los métodos automáticos se asocian con mejor retención.

Sugerencia: Activar débito/autopay con un incentivo.

4) InternetService

Dentro de los clientes que tienen internet, fiber optic retiene peor que DSL. Esto probablemente se deba al precio y competencia de ese segmento. 

Sugerencia: Revisar la relación pricing-beneficios para fiber optic y comunicar el valor que lo distingue.

5) OnlineSecurity

Contar con seguridad online se asocia a más permanencia. 

Sugerencia: empaquetar la seguridad como beneficio de fidelidad y facilitar su adopción en clientes de riesgo.

6) TechSupport

Tener soporte técnico también coincide con mayor permanencia.

Sugerencia: mantener un contacto proactivo con clientes de alto riesgo (sin soporte contratado)

7) MultipleLines

La señal es más tenue: no parece un driver principal por sí solo.

### 2.3 Cargos mensuales por servicio

In [ ]:
cats = [c for c in df_model["InternetService"].dropna().unique().tolist()]
data = [df_model.loc[df_model["InternetService"]==c, "MonthlyCharges"].dropna().values for c in cats]
plt.figure()
plt.boxplot(data, labels=cats, showmeans=True)
plt.title("MonthlyCharges por InternetService")
plt.xlabel("InternetService"); plt.ylabel("MonthlyCharges")
plt.tight_layout(); plt.show()

#### Conclusiones

Fiber optic aparece con las facturas mensuales más altas; DSL significativamente más bajo; sin internet, muy bajo. Esto conecta con lo visto anteriormente: precio alto + fibra = mayor riesgo. 


### 2.5 Matriz de correlación

In [ ]:
# Paso 1: Seleccionar columnas numéricas
num_df = df_model.select_dtypes(include=["int64", "float64"])

# Paso 2: Identificar columnas binarias (solo contienen 0 y 1)
binary_cols = [col for col in num_df.columns 
               if set(num_df[col].dropna().unique()) <= {0, 1}]

# Paso 3: Excluir columnas binarias
continuous_cols = [col for col in num_df.columns if col not in binary_cols]

# Paso 4: Calcular la matriz de correlación solo con variables continuas
corr_matrix = df_model[continuous_cols].corr()

# Paso 5: Visualizar
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de Correlación - Solo Variables Continuas")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

#### Conclusiones

La señal dominante de permanencia es la antigüedad; después pesan variables ligadas a costo y a tener internet. También se observa una leve relación negativa con SeniorCitizen, que conviene monitorear por equidad.


## ⚙️ Plan de análisis

### Paso 1: Integración y limpieza
✔ Unir `contract`, `personal`, `internet` y `phone` por `customerID`
✔ Conversión de tipos de datos y fechas  
✔ Calcular `tenure_months` al 01-feb-2020
✔ Normalizar categorías (Yes/No)
✔ Crear las etiquetas `stay` (objetivo) y `churn` (referencia)

### Paso 2: EDA y chequeos de consistencia
✔ Explorar nulos, distribuciones y tasas de **stay** por segmento   
✔ Verificar coherencia entre `HasInternet/HasPhone` y las columnas asociadas
✔ Crear gráficos para visualización de datos

### Paso 3: Preparación de features y partición
✔ Definir variables numéricas/categóricas, aplicar One-Hot a categóricas y escalado a numéricas con `ColumnTransformer`  
✔ Hacer `train/test` (temporal o estratificado)  
✔ Preparar estrategia ante desbalance (si aplica)

### Paso 4: Modelado baseline y evaluación
✔ Entrenar modelos base y evaluar con **AUC-ROC** (principal) y **Accuracy** (adicional) usando la probabilidad de la clase **1 = stay* 

## Paso 5: Entrega
✔ Generar un csv limpio, guardar notebook reproducible y un breve reporte con métricas y hallazgos

✔ Definir mejoras y plan de monitoreo


## ❓ Preguntas clave

### 1) Negocio y objetivo
- **Población objetivo:** ¿Modelamos a **todos** los clientes o solo a quienes cumplan cierta antigüedad/plan?
- **Política de recontacto:** ¿Cada cuánto se reintenta a un mismo cliente?

### 2) Datos y etiqueta
- **Definición de stay/churn:** ¿EndDate = "No" ≡ contrato realmente activo? ¿Existen suspensiones?
- **Unidad de análisis:** ¿Cliente individual, cuenta/hogar o línea?
- **Churn parcial:** Si cancela internet pero conserva phone (o viceversa), ¿cómo se etiqueta?
- **Corte temporal:** ¿Todas las variables se toman al **01-feb-2020** (sin fugas)?
- **Promos previas:** ¿Existen variables de campañas pasadas que puedan introducir fuga?

### 3) Métricas y validación
- **Desbalance:** ¿class_weight u oversampling? (que no distorsione la evaluación)
- **Métricas por segmento:** ¿Reportamos por contrato, método de pago, senioridad, etc.?

### 4) Features e ingeniería
- **Variables sensibles:** ¿`gender` solo para monitoreo de sesgo (no para decisión)?
- **Transformaciones:** One-hot para categóricas, escalado para numéricas, ¿outliers?
- **Faltantes:** ¿Aceptamos `TotalCharges = MonthlyCharges * tenure_months` o preferimos otra política?

### 5) Operación e intervención
- **Consumo del score:** ¿Batch diario/semanal? ¿Formato de entrega (CSV/API/Dashboard)?
- **Umbral:** Aunque la rúbrica use AUC-ROC, ¿qué *threshold* se aplicará en producción?
- **Medición:** ¿Habrá grupo de control/AB test? 


---

> **Transición**: A continuación se incluye la segunda parte del proyecto en el mismo cuaderno.

## 📣 Pronóstico de tasa de cancelación de clientes - Telecom

# 📣 Pronóstico de tasa de cancelación de clientes - Telecom

## 📌 Introducción
Al operador de telecomunicaciones Interconnect le gustaría poder pronosticar su tasa de cancelación de clientes. 
    
Si se descubre que un usuario o usuaria planea irse, se le ofrecerán códigos promocionales y opciones de planes especiales. 
 

## 🎯 Problema de negocio
La clientela puede elegir entre un pago mensual o firmar un contrato de 1 o 2 años. Puede utilizar varios métodos de pago y recibir una factura electrónica después de una transacción. Telecom necesita descubrir si un usuario o usuaria planea irse, para ofrecerle códigos promocionales y opciones de planes especiales. 


## 🔍 Objetivo del proyecto
✔ Preparar un *snapshot* al **01-feb-2020** y un dataset limpio para modelado donde la **clase positiva** sea **`stay = 1`** (cliente activo, `EndDate == "No"`).  


## 📝 Proceso de análisis
- Entrenar y comparar modelos (Logística, RandomForest, HistGradientBoosting).
- Calibrar probabilidades y evaluar: **AUC-ROC (stay)**, **Accuracy**, **PR-AUC (churn)**.
- Seleccionar **umbral** y generar reporte **Top-k**.
- Priorizar por **riesgo × valor (ERV / NetERV)** usando columnas disponibles.
- Exportar CSVs operativos y el pipeline.

## Inicialización

In [ ]:
# Cargar todas las librerías

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    roc_auc_score, accuracy_score, average_precision_score, precision_score, recall_score, f1_score,
    precision_recall_curve, confusion_matrix, classification_report)

import joblib

In [ ]:
# Parámetros de negocio 
HORIZON_M2M = 12 # meses para contratos Month-to-month
MARGIN_RATE = 0.30 # margen aproximado sobre facturación
SUCCESS_RATE = 0.25 # tasa de éxito esperada de la campaña (para NetERV)
COST_PER_CONTACT = 1.5 # costo por contacto (llamada/SMS/email)

RANDOM_STATE = 42 #seeds

In [ ]:
df = pd.read_csv("./datasets/data_clean.csv")

In [ ]:
df.info()

In [ ]:
df.head()

## 📌 1. Definir objetivo y tipos de variables

In [ ]:
# Objetivo: preferimos 'stay'. Si no existe, usamos 1 - churn.
cols_lower = {c.lower(): c for c in df.columns}

y_col = None
if 'stay' in df.columns:
    y_col = 'stay'
elif 'churn' in df.columns:
    y_col = 'churn'

assert y_col is not None, "No se encontró columna objetivo ('stay' o 'churn')."

In [ ]:
y_raw = df[y_col].copy()
if y_col.lower() == 'churn':
    target = (1 - y_raw).astype(int)
    print("Usando 'churn' invertido como objetivo (stay=1).")
else:
    target = y_raw.astype(int)
    print("Usando 'stay' como objetivo.")

In [ ]:
f_drop = set([y_col, 'stay', 'churn', 'customerID', 'customerid'])
features = df[[c for c in df.columns if c not in f_drop]].copy()

# Detectar tipos
num_cols = features.select_dtypes(include=['number']).columns.tolist()
cat_cols = features.select_dtypes(exclude=['number']).columns.tolist()

print("Variables numéricas detectadas:", num_cols[:12], "... total:", len(num_cols))
print("Variables categóricas detectadas:", cat_cols[:12], "... total:", len(cat_cols))


## ✏️ 2. División en conjunto de entrenamiento y prueba

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, stratify=target, random_state= RANDOM_STATE
)
print("Train:", features_train.shape, " Test:", features_test.shape)

## 🚩 3. Preprocesamiento y comparación de modelos

### 3.1 Preprocesamiento para Regresión Logística

In [ ]:
pre_lr = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

### 3.2 Preprocesamiento para Árboles 

In [ ]:
pre_tree = ColumnTransformer([
    ("num", "passthrough", num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

### 3.3 Comparación de modelos candidatos

In [ ]:
candidates = {
    "logreg": Pipeline([("pre", pre_lr),
                        ("clf", LogisticRegression(max_iter=500, class_weight="balanced", random_state=RANDOM_STATE))]),
    "rf": Pipeline([("pre", pre_tree),
                    ("clf", RandomForestClassifier(n_estimators=300, min_samples_leaf=2, n_jobs=-1, random_state=RANDOM_STATE))]),
    "hgb": Pipeline([("pre", pre_tree),
                     ("clf", HistGradientBoostingClassifier(random_state=RANDOM_STATE))])
}


### 3.4 Validación cruzada estratificada y selección por AUC-ROC

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_scores = {}
for name, pipe in candidates.items():
    scores = cross_val_score(pipe, features_train, target_train, scoring="roc_auc", cv=cv, n_jobs=-1)
    cv_scores[name] = (scores.mean(), scores.std())
    print(f"{name}: ROC AUC (cv) = {scores.mean():.4f} ± {scores.std():.4f}")

best_name = max(cv_scores, key=lambda k: cv_scores[k][0])
best_pipe = candidates[best_name]
print("\nMejor (cv):", best_name, "->", cv_scores[best_name])

## 🔨 4. Entrenamiento y prueba de modelo Histogram-based Gradient Boosting

In [ ]:
best_pipe.fit(features_train, target_train)
p_stay_test = best_pipe.predict_proba(features_test)[:, 1]
target_hat_test  = (p_stay_test >= 0.5).astype(int)

auc = roc_auc_score(target_test, p_stay_test)
acc = accuracy_score(target_test, target_hat_test)
print(f"AUC-ROC(stay)={auc:.4f}  Accuracy={acc:.4f}")

## ❗ 5. PR-AUC (churn) y tabla de umbrales

In [ ]:
p_churn_use = 1 - p_stay_test
target_test_churn = 1 - target_test

pr_auc = average_precision_score(target_test_churn, p_churn_use)
print(f"PR-AUC (churn): {pr_auc:.4f}")


### 5.1 Tabla de umbrales

In [ ]:
prec, rec, thr = precision_recall_curve(target_test_churn, p_churn_use)
f1 = 2 * (prec * rec) / (prec + rec + 1e-12)          
thr_grid = np.append(thr, 1.0)                         

thr_table = pd.DataFrame({
    "threshold_churn": thr_grid,
    "precision": prec,
    "recall": rec,
    "F1": f1
}).sort_values("threshold_churn").reset_index(drop=True)

display(thr_table.head(10))
display(thr_table.tail(10))

#### 5.1.1 Umbral que maximiza F1

In [ ]:
best_idx = np.nanargmax(thr_table["F1"].values)
thr_f1 = float(thr_table.loc[best_idx, "threshold_churn"])
target_pred_f1 = (p_churn_use >= thr_f1).astype(int)

print(f"\nUmbral óptimo por F1 (churn): {thr_f1:.4f}")
cm = confusion_matrix(target_test_churn, target_pred_f1)
print("Matriz de confusión (umbral F1):\n", cm)


In [ ]:
# Métricas al corte 
prec_val = precision_score(target_test_churn, target_pred_f1, zero_division=0)
rec_val = recall_score(target_test_churn, target_pred_f1, zero_division=0)
f1_val = f1_score(target_test_churn, target_pred_f1, zero_division=0)
print("Métricas en TEST (umbral F1):",
      "Precision=", round(prec_val, 3),
      "Recall=",    round(rec_val, 3),
      "F1=",        round(f1_val, 3))

print("\nReporte de clasificación (umbral F1):\n",
      classification_report(target_test_churn, target_pred_f1, digits=3))

## ✔️ 6. Valor del cliente y ERV / NetERV 

In [ ]:
def find_col(candidates):
    lower = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand in df.columns:
            return cand
        if cand.lower() in lower:
            return lower[cand.lower()]
    return None

col_monthly = find_col(['MonthlyCharges','monthly_charges','mensual','monto_mensual'])
col_total   = find_col(['TotalCharges','total_charges','facturacion_total','acumulado'])
col_type    = find_col(['Type','Contrato','contract_type','tipo'])
col_tenure  = find_col(['tenure_months','tenure','antiguedad_meses'])

print("Detección columnas valor -> Monthly:", col_monthly, "| Total:", col_total, "| Type:", col_type, "| Tenure:", col_tenure)

df_scores = df.copy()
idx_test = getattr(features_test, 'index', pd.RangeIndex(len(target_test)))
df_scores = df_scores.loc[idx_test].copy()
df_scores["p_churn"] = p_churn_use
df_scores["target_true_churn"] = (1 - target_test.values)

# Construcción de valor con degradación elegante
if col_monthly is not None:
    monthly = df[col_monthly]
else:
    # Fallback: si no hay monthly, aproximar con Total/ max(tenure,1) limitado [1, 100] (muy bruto)
    if (col_total is not None) and (col_tenure is not None):
        monthly = (df[col_total] / np.clip(df[col_tenure].replace(0,1), 1, None)).fillna(df[col_total].median()).clip(lower=1)
    else:
        monthly = pd.Series(np.full(len(df), df[col_total].median() if col_total else 50.0))

# Horizonte por tipo
if col_type is not None:
    def rem_months(row):
        t = row[col_type]
        ten = row[col_tenure] if col_tenure in df.columns else 0
        if str(t).lower().startswith("one") or str(t).lower().startswith("1"):
            return max(12 - (int(ten) % 12), 0)
        if str(t).lower().startswith("two") or str(t).lower().startswith("2"):
            return max(24 - (int(ten) % 24), 0)
        return HORIZON_M2M
    rem = df.apply(rem_months, axis=1)
else:
    rem = pd.Series(np.full(len(df), HORIZON_M2M))

value_forward = monthly * rem
value_forward_margin = value_forward * MARGIN_RATE

df_scores["Value_forward_margin"] = value_forward_margin.loc[idx_test].values
df_scores["ERV"] = df_scores["p_churn"] * df_scores["Value_forward_margin"]
df_scores["NetERV"] = df_scores["p_churn"] * df_scores["Value_forward_margin"] * SUCCESS_RATE - COST_PER_CONTACT

display(df_scores[["p_churn","target_true_churn","Value_forward_margin","ERV","NetERV"]].head())

## 📝 7. Reporte de Top-k

In [ ]:
def topk_report(df_scores, k_list=(5,10,20,30), use_net=False):
    metric_value = "NetERV" if use_net else "ERV"
    out = []
    n = len(df_scores)
    df_sorted = df_scores.sort_values("p_churn", ascending=False).reset_index(drop=True)
    total_churn = df_sorted["target_true_churn"].sum()
    total_value = df_sorted[metric_value].sum()
    for k in k_list:
        cut = int(np.ceil(n * k/100))
        take = df_sorted.iloc[:cut]
        captured_churn = take["target_true_churn"].sum()
        captured_value = take[metric_value].sum()
        out.append({
            "top_k_%": k,
            "contactados": cut,
            "churn_capturado_%": round(100 * captured_churn / max(total_churn,1), 1),
            f"valor_capturado_%({metric_value})": round(100 * captured_value / max(total_value,1), 1),
            "precision_en_topk": round(take["target_true_churn"].mean(), 3),
            f"{metric_value}_total_topk": round(float(captured_value), 2)
        })
    return pd.DataFrame(out)

report_topk_erv = topk_report(df_scores, k_list=(5,10,20,30), use_net=False)
report_topk_net = topk_report(df_scores, k_list=(5,10,20,30), use_net=True)

print("Top-k por ERV:"); display(report_topk_erv)
print("Top-k por NetERV:"); display(report_topk_net)



## 🗂️ 7. Ranking operativo

In [ ]:
def find_col(df, candidates):
    lower = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand in df.columns:
            return cand
        if cand.lower() in lower:
            return lower[cand.lower()]
    return None

id_col = find_col(df, ['customerID','CustomerID','id','Id'])
col_type = find_col(df, ['Type','Contrato','contract_type','tipo','Contract','ContractType'])
col_monthly= find_col(df, ['MonthlyCharges','monthly_charges','monto_mensual'])
col_tenure = find_col(df, ['tenure_months','tenure','antiguedad_meses'])

cols_rank = [c for c in [id_col, col_type, 'PaymentMethod', col_monthly, col_tenure] if c and c in df.columns]

# Construir el ranking (p_churn, ERV, NetERV ya están en df_scores)
to_join = cols_rank  # puede estar vacío; no pasa nada
ranking = (
    df_scores[['p_churn','ERV','NetERV']]
    .join(df.loc[df_scores.index, to_join], how='left')
    .sort_values(['p_churn','ERV'], ascending=[False, False])
    .reset_index(drop=True)
)

display(ranking.head(20))



## 📤 8. Exportar CSVs y guardar pipeline

In [ ]:
# Exportar CSVs
out_dir = Path("outputs"); out_dir.mkdir(exist_ok=True)
report_topk_erv.to_csv(out_dir / "reporte_topk_ERV.csv", index=False)
report_topk_net.to_csv(out_dir / "reporte_topk_NetERV.csv", index=False)
ranking.to_csv(out_dir / "ranking_operativo.csv", index=False)
print("Exportados: outputs/reporte_topk_ERV.csv, outputs/reporte_topk_NetERV.csv, outputs/ranking_operativo.csv")

joblib.dump(best_pipe,"pipeline.joblib")
print("Guardado: pipeline.joblib")

##  Resumen ejecutivo
- **Objetivo de negocio:** Identificar y priorizar clientes con alta probabilidad de churn y alto valor esperado. 
- **Métricas (TEST):** ROC‑AUC (stay)= 0.9172, PR‑AUC (churn)= 0.8636  
- **Política de decisión:** Umbral F1 = 0.446 (máx-F1 en TEST). o, si prefieren operar por presupuesto, Top-k = 10% priorizando por NetERV.  
- **Impacto esperado:** con Top-10% se capturó ≈ 37.7% de churn y 38.1% del valor (NetERV).  
- **Entregables:** pipeline `.joblib` + ranking operativo `.csv` + tablas Top‑k.

---

##  Problema y alcance
- Definición de **churn/stay**, unidad de análisis y horizonte.  
- Criterios de éxito (métrica principal y secundaria).  
- Suposiciones y exclusiones (qué no modelamos y por qué).

---

##  Datos y etiqueta
- Fuentes: `contract.csv`, `personal.csv`, `internet.csv`, `phone.csv`.  
- Etiqueta: convención **stay=1** (si había `churn`, se usó `1 - churn`).  
- Prevención de leakage: columnas excluidas (p.ej., `EndDate`, IDs, estados post‑corte).

---

##  EDA (hallazgos accionables)
- Distribución de clases, NAs relevantes, outliers.  
- Señales visibles: tipo de contrato, tenure, cargos, add‑ons, etc.  
- (Incluye 2–4 gráficos clave con una frase de insight cada uno.)

---

##  Preprocesamiento y features
- Limpiezas: `TotalCharges` → `MonthlyCharges × tenure` cuando faltó; normalización Yes/No.  
- Transformaciones: One‑Hot (categóricas); escalado **solo** para logística.  
- Features clave: `months_left`, `value_forward_est`, `addons_count`, `mc_x_tenure`, etc.

---

##  Modelado y validación
- Pipelines con `ColumnTransformer` (sin fugas).  
- Candidatos: LogReg (balanced), RandomForest, HistGradientBoosting.  
- Validación: StratifiedKFold(5), `scoring='roc_auc'` (principal).

---

##  Resultados (TEST)
- ROC‑AUC (stay) = 0.9172; PR‑AUC (churn) = 0.8636.  
- Tabla de umbrales (precision/recall/F1); **corte elegido** = 0.4014.  
- Matriz de confusión al corte y métricas al corte.  
- (Opcional) Calibración isotónica: usada / no usada (motivo).

---

##  Operación y valor
- Definiciones: **ERV = p_churn × valor_margen**; **NetERV = ERV × tasa_éxito − costo**.  
- Parámetros: margen=0.30, tasa_éxito=0.25, costo_contacto=1.5, horizonte M2M=12 meses.  
- Top‑k (5/10/20/30): precisión, recall, % valor capturado.  
- Ranking operativo exportado: ruta de archivos y columnas incluidas.

---

##  Reproducibilidad
- Semilla fija y dónde se aplica (split, CV, modelos).  
- Versiones de Python/librerías.  
- Cómo ejecutar end‑to‑end en 3–5 pasos.  
- Ubicación de artefactos y outputs.

---

##  Riesgos, límites y próximos pasos
- Límites de señal (no hay uso/pagos/soporte históricos); posibles mejoras.  
- Siguientes pasos: integrar nuevas fuentes, tuning adicional / CatBoost, interpretabilidad (SHAP), monitoreo y retraining.

# Informe detallado

## ✅ 1. Pasos realizados

### EDA y preprocesamiento
Se unieron las tablas contract/personal/internet/phone. Se preparó una limpieza de TotalCharges y se normalizaron las categorías yes/no. Se creó la columna tenure_months y se corrigió TotalCharges ≈ MonthlyCharges × tenure cuando faltó. Además se excluyeron las columnas con riesgo de leakage (EndDate, IDs).

### Definición de objetivo
Se trabajó con la convención stay=1. Si venía churn, se usó "y = 1 - churn" para mantener "1 = se queda."

### Características
Se separaron las variables en numéricas/categóricas. Después se aplicó one-hot a las categóricas con un escalado solo para logística.

### Partición estratificada y pipelines con ColumnTransformer
Se realizó un train_test_split estratificado. Además, se realizó una comparación entre los modelos de Regresión Logística, RandomForest y HistGradientBoosting (HGB) con StratifiedKFold (5 folds) y ROC-AUC como métrica principal.

### Entrenamiento de modelo
Se realizó el entrenamiento del modelo del mejor pipeline y se evaluó en test (ROC-AUC y PR-AUC).

### Curvas PR (churn) y tabla de umbrales (precision/recall/F1)
Se generaron curvas PR (churn), tabla de umbrales (precision/recall/F1), y se hizo una selección de corte por máx-F1 (además de opción por recall objetivo).

### Estrategias de operación
Top-k por riesgo y priorización económica (ERV/NetERV), con exportables (ranking_operativo_*.csv, reporte_topk_*.csv), y guardado del pipeline con joblib.

## 🚩 2. Pasos omitidos

### Validación temporal: 
No hay timestamps detallados para corte temporal; se usó CV estratificado clásico.


## ⛔ 3. Dificultades y solución

- **Desbalance natural (más stay que churn):** se usó class_weight="balanced" en logística y se reportó ROC-AUC (ranking global) y PR-AUC (churn) (calidad en la clase minoritaria).

- **Heterogeneidad de nombres:** helpers para encontrar columnas (find_col) y evitar quiebres si cambia capitalización o idioma.

- **Probabilidades no calibradas:** se prefiere Top-k o umbrales “de negocio” (no probabilidades absolutas).

- **Riesgo vs. Valor:** se añadió ERV/NetERV para priorizar a quién contactar maximizando retorno esperado, no solo riesgo.

## 🗝️ 4. Pasos clave para resolver la tarea

- **Pipelines con ColumnTransformer** (sin fugas): one-hot a categóricas y escalado sólo donde aporta (logística).
- **Selección por CV (AUC-ROC)** para elegir el mejor algoritmo sin mirar _test_.
- **PR-AUC + Umbrales/Top-k** para pasar de métricas a decisiones.
- **Riesgo × Valor** (**ERV/NetERV**) para priorizar contactos con mayor impacto económico esperado.

## ☑️ 5. Modelo final y nivel de calidad

**Modelo final:** Pipeline con HistGradientBoostingClassifier (HGB) (mejor por CV en esta corrida).

**Calidad (TEST):**

 - ROC-AUC (stay): 0.9172

 - PR-AUC (churn): 0.8636
(Los valores pueden moverse levemente si cambias la semilla o reentrenas.)